In [ ]:
from aux_functions import tqa_data_extraction, tqa_data_refiner, get_tqa_model, get_tqa_sequence_lengths, get_dropouts
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, precision_score, recall_score
from keras import optimizers
from sklearn.metrics import classification_report

## Experiment selection

In [ ]:
#question_type = "diagramQuestions"
question_type = "nonDiagramQuestions"

#weights = "cross"
weights = "cross-vecsi"

print("Selected the "+question_type+" corpus and weights from " +weights)

## Input files and parameters

In [ ]:
train_folder = "./tqa_train_val_test/train/"
train_json = "./tqa_train_val_test/train/tqa_v1_train.json"
test_folder = "./tqa_train_val_test/test/"
test_json = "./tqa_train_val_test/test/tqa_v2_test.json"
val_folder = "./tqa_train_val_test/val/"
val_json = "./tqa_train_val_test/val/tqa_v1_val.json"

dataset_jsons = [train_json,test_json,val_json]
dataset_folders = [train_folder,test_folder,val_folder]

dim = 100
epochs = 10
batchSize = 128
dout, rdout = get_dropouts(question_type)
context_length, question_length = get_tqa_sequence_lengths(question_type)

## Loading corpus

In [ ]:
data, figures, correct_answers = tqa_data_extraction(dataset_folders, dataset_jsons, question_type)

#Data_refining
texts = []

for i in range(len(data[0])):
    for j in range(len(data)):
        texts.append(data[j][i])

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# Get the vocabulary index
word_index = tokenizer.word_index
print("Found %s unique tokens." % len(word_index))

x, y = tqa_data_refiner(data, figures, correct_answers,tokenizer, weights, context_length, question_length)

## Training model (cross-validation)

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)
precisions = []
recalls = []
f1s = []
n_words = len(word_index)

for train, test in kfold.split(x[0], y):
    
    x_train = []
    x_test = []
    for elem in x:
        x_train.append(elem[train])
        x_test.append(elem[test])
    y_train = y[train]
    y_test = y[test]
    
    model = get_tqa_model(n_words, dim, dout, rdout, context_length, question_length)
    
    adam = optimizers.Adam(lr=1e-2, decay=0.0)
    model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=['categorical_accuracy'])
    model.fit(x_train, y_train, batch_size=batchSize, epochs=epochs, validation_data=(x_test, y_test), verbose=1)
    
    pred = model.predict(x_test, batch_size=batchSize)
    max_value = np.argmax(pred,axis=1)
    predNew = np.zeros(np.shape(pred))
    for i in range(len(predNew)):
        predNew[i,max_value[i]]=1
    print(classification_report(y_test, predNew, digits=4, target_names=["a","b","c","d"]))
    precisions.append(precision_score(y_test, predNew, average="weighted"))
    recalls.append(recall_score(y_test, predNew, average="weighted"))
    f1s.append(f1_score(y_test, predNew, average="weighted"))
    
print("Precision: %.2f (+/- %.2f)" % (np.mean(precisions), np.std(precisions)))
print("Recall: %.2f (+/- %.2f)" % (np.mean(recalls), np.std(recalls)))
print("F1 Score: %.2f (+/- %.2f)" % (np.mean(f1s), np.std(f1s)))
    